In [1]:
%%capture
import sys

!python -m pip install TextBlob
!python -m pip install newspaper3k
!python -m pip install preprocessor
!python -m pip install Twython
!python -m pip install tweepy
!python -m pip install os
!python -m pip install nltk
!python -m pip install tensorflow
!python -m pip install contractions
!python -m pip install autocorrect
!python -m pip install betterspy
!python -m pip install tensorflow
!python -m pip install cvxopt import matrix, solvers

In [2]:
# Load packages
import os
import time

from textblob import TextBlob
from newspaper import Article
import nltk
import re

import requests
import preprocessor as pre
import csv
import json
import string
from string import punctuation
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from contractions import contractions_dict
from autocorrect import Speller
import seaborn as sns
import pylab as pl
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

import numpy as np
import pandas as pd
import sklearn
import tensorflow

import tweepy
from tweepy import OAuthHandler
from twython import Twython

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Data Setup

In [3]:
# Handling directories
from google.colab import drive
drive.mount('/content/drive')
df_movie = pd.read_csv('/content/drive/My Drive/Spring 2022/CS4824/finalproj/movies/movie.csv')

#Append the directory to your python path
prefix = '/content/drive/My Drive/'
# modify customized_path
customized_path = 'Spring 2022/CS4824/finalproj/'
data_folder = 'movies/'
sys_path = prefix + customized_path + data_folder
sys.path.append(sys_path)
print(sys.path)

data_filename = os.path.join(sys_path, 'movie.csv')
print('Path to training data: {}'.format(data_filename))
#fn_test = os.path.join(sys_path, './')
#print('Path to testing data: {}'.format(fn_test))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/Spring 2022/CS4824/finalproj/movies/']
Path to training data: /content/drive/My Drive/Spring 2022/CS4824/finalproj/movies/movie.csv


In [4]:
#Loading in and splitting training and test data
df_orig = pd.read_csv(data_filename)
df_orig = df_orig.drop(labels=range(20001, 40000))

print(df_orig)

                                                    text  label
0      I grew up (b. 1965) watching and loving the Th...      0
1      When I put this movie in my DVD player, and sa...      0
2      Why do people who do not know what a particula...      0
3      Even though I have great interest in Biblical ...      0
4      Im a die hard Dads Army fan and nothing will e...      1
...                                                  ...    ...
19996  "White Noise" had potential to be one of the m...      0
19997  The Five Deadly Venoms is a great kung-fu acti...      1
19998  Ali G Indahouse has got to be one of the funni...      1
19999  I found myself at sixes and sevens while watch...      1
20000  Christopher Lambert is annoying and disappoint...      0

[20001 rows x 2 columns]


# Text Preprocessing

In [5]:
def lowercase(text):
  """
    Converts text to be completely lowercase.

    Argument: String of text.
    Returns: Completely lowercase version of input text.
  """
  return text.lower()
  
def remove_stopwords(text):
  """
    Removes all stopwords from the text.
    Note: Refers to nltk's default English stopword list.

    Argument: String of text.
    Returns: Input text, now free of stopwords.
  """
  #Treat text as sentence
  stopword_list = stopwords.words('english')
  return ' '.join([word for word in nltk.word_tokenize(text) if not word in stopword_list])

def lemmatize(text_tagged):
  """
    Normalizes all words in the text
    Note: Utilizes nltk's WordNet's lemmatizer to derive lemmatizations.

    Argument: A list of tuples: [(<word>, <part of speech retag>),...]
    Returns: Lemmatized form of text
  """
  lemmatizer = WordNetLemmatizer()
  lemmatized = ""
  for word, pos in text_tagged:
    if not pos:
      lemmatized = lemmatized + " " + word
    else:
      lemmatized = lemmatized + " " + lemmatizer.lemmatize(word, pos=pos)

  return lemmatized

def retag(text_tagged):
  """
    Rewrites the part of speech tags for each word into a form
    interpretable by WordNet.

    Argument: A list of tuples: [(<word>, <part of speech tag>),...]
    Returns: Lemmatized form of text
  """
  pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
  retagged = []
  for word, tag in text_tagged:
      retagged.append(tuple([word, pos_dict.get(tag[0])]))

  return retagged

def spellcheck(text):
  corrections = [Speller.spell(word) for word in nltk.word_tokenize(text)]
  return ' '.join(corrections)


#All functions combined
def preprocessing3(text):
  """
    Aggregation of preprocessing functions into one.
    Note: Additionally uses re's sub() function for convenient cleaning.

    Argument: Text, as a string.
    Returns: Preprocessed text.
  """
  lower = lowercase(text)
  cleaned = re.sub('[^A-Za-z]+', ' ', lower)
  stopwordfree = remove_stopwords(cleaned)
  tokenized = word_tokenize(stopwordfree)
  pos = nltk.pos_tag(tokenized)
  posretagged = retag(pos)
  lemmatized = lemmatize(posretagged)
  tokenized2 = word_tokenize(lemmatized)
  
  #return tokenized2 #final, option1
  return lemmatized #final, option2

In [6]:
extext = df_orig.iloc[50]['text']

print(extext)
print(preprocessing3(extext))

When this first came out, my dad brought it home- we were amazed by it- It was so different from anything we had seen before. I was looking for a specific movie last night, and I found "The Mind's Eye" again. The box is falling apart, and I am surprised that the tape still works! Although it is not 'Finding Nemo' quality graphics, it is still very good. They should sell this again- it is a landmark for computer animation imagery. Highly recommended!<br /><br />This is what it is:<br /><br />"The Mind's Eye" is a spectacular odyssey through time. Your journey begins at the dawn of creation and moves through the rise of man and technology. Travel in the world of abstraction and on into the future with breathtaking computer animation imagery.<br /><br />"The Mind's Eye" joins the imaginations of over 300 of the world's most talented computer animation artists with a powerful, original music soundtrack. This unique collaboration takes you on an incredible voyage into "The Mind's Eye."
 fir

In [7]:
df_labels = df_orig['label']
df_text = df_orig['text'].apply(preprocessing3, 'expand')

print(df_text.head())

0     grow b watch loving thunderbird mat school wa...
1     put movie dvd player sit coke chip expectatio...
2     people know particular time past like feel ne...
3     even though great interest biblical movie bor...
4     im die hard dad army fan nothing ever change ...
Name: text, dtype: object


# Training and Test Split

In [8]:
data_proc = {'text': df_text, 'label': df_labels}
df_proc = pd.DataFrame(data=data_proc)
print(df_proc)

from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_proc, test_size=0.2)

train_data = train_df['text']
train_labs = train_df['label']
test_data = test_df['text']
test_labs = test_df['label']

                                                    text  label
0       grow b watch loving thunderbird mat school wa...      0
1       put movie dvd player sit coke chip expectatio...      0
2       people know particular time past like feel ne...      0
3       even though great interest biblical movie bor...      0
4       im die hard dad army fan nothing ever change ...      1
...                                                  ...    ...
19996   white noise potential one talk movie since ex...      0
19997   five deadly venom great kung fu action movie ...      1
19998   ali g indahouse get one funny film see long t...      1
19999   find six sevens watch one altman touch zoom e...      1
20000   christopher lambert annoy disappointing portr...      0

[20001 rows x 2 columns]


# Text Vectorization

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=100)
vectorizer.fit(df_proc['text'])

train_data_tfidf = vectorizer.transform(train_data)
test_data_tfidf = vectorizer.transform(test_data)

In [10]:
print(len(vectorizer.vocabulary_))
print(type(train_data_tfidf))
print(train_data_tfidf.shape)
print(train_data_tfidf[0])
print(np.max(train_data_tfidf))

Xtrain = train_data_tfidf.toarray()
Xtest = test_data_tfidf.toarray()
ytrain = train_labs.to_numpy()
ytest = train_labs.to_numpy()

print(Xtrain.shape)
print(Xtrain)
print(ytrain.shape)
print(ytrain.T)

100
<class 'scipy.sparse.csr.csr_matrix'>
(16000, 100)
  (0, 96)	0.20191785489033795
  (0, 95)	0.2982031584782163
  (0, 92)	0.18847432307568485
  (0, 74)	0.28736025576726243
  (0, 55)	0.14718939284678614
  (0, 49)	0.13604868275388562
  (0, 46)	0.26742195117252415
  (0, 45)	0.15722639045956285
  (0, 42)	0.21790997999208273
  (0, 39)	0.16318711066953273
  (0, 36)	0.22022317642833636
  (0, 31)	0.19274762015740782
  (0, 24)	0.1449538840701364
  (0, 18)	0.22115012331190595
  (0, 10)	0.5844066055047122
  (0, 7)	0.21114497090463594
1.0
(16000, 100)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.13081209 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.25488511 ... 0.         0.         0.        ]
 ...
 [0.20897456 0.         0.         ... 0.         0.23248711 0.        ]
 [0.         0.20909471 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.32442026 0.        ]]
(16000,)

## Feature Selection

In [11]:
print(np.count_nonzero(Xtrain[1,:])/len(Xtrain[1,:]))
print(train_data_tfidf[1,:].getnnz()/80506)

print(train_data_tfidf[1035,:])
print(train_data_tfidf[:,1])
#print(np.dot(train_data_tfidf[1,:], train_data_tfidf[5,:]))

0.23
0.0002856929918266962
  (0, 1)	0.15502356529668732
  (0, 3)	0.1414281612393872
  (0, 11)	0.17835585619916994
  (0, 12)	0.12648820566978783
  (0, 16)	0.18401945522688842
  (0, 18)	0.13918826313821095
  (0, 24)	0.3649263958203485
  (0, 28)	0.11151593117851612
  (0, 36)	0.27720971592322613
  (0, 37)	0.18070163825583493
  (0, 38)	0.156541208194308
  (0, 41)	0.18418668858127107
  (0, 44)	0.1472762999944608
  (0, 45)	0.19791142667998296
  (0, 48)	0.18834732353338032
  (0, 51)	0.15525909511405214
  (0, 52)	0.18430642791196367
  (0, 55)	0.09263859154090294
  (0, 61)	0.18354420619710285
  (0, 67)	0.13787248421020032
  (0, 68)	0.14034943343506187
  (0, 69)	0.10214632448070711
  (0, 70)	0.1539894486853819
  (0, 73)	0.1678722887570896
  (0, 75)	0.1821869621228259
  (0, 80)	0.14409692043747144
  (0, 81)	0.1296617086259546
  (0, 86)	0.15751349926443134
  (0, 92)	0.1186226499998623
  (0, 93)	0.18652864293477617
  (0, 96)	0.25416757719046135
  (0, 99)	0.18608248422986853
  (4, 0)	0.13729169428986

In [12]:
import math 

#print(train_data[0])

def txf(sentence):
  word_dict = {}
  words = word_tokenize(sentence);
  
  for word in words:
    word_dict[word] = word_dict.get(word, 0) + 1
  return word_dict

print(txf(extext))
print(len(df_orig))

class TFIDF:
  def __init__(self):
    self.word_index = {}
    self.index_word = {}
    self.idf_dict = {}
  
  def fit(self, data):
    text_list = data
    text_num = len(text_list)

    global_tf = {}
    for text in text_list:
      wordset = set()
      text_tf = txf(text)

      for word in text_tf:
        if word not in wordset:
          global_tf[word] = global_tf.get(word,0) + 1
          wordset.add(word)

    for word, freq in global_tf.items():
      idf = math.log((1+len(data)) / (1+freq))
      self.idf_dict[word] = idf

    text_words = list(global_tf.keys())
    for i in range(len(text_words)):
      word = text_words[i]
      self.word_index[word] = i
      self.index_word[i] = word

    print(type(global_tf.items()))

  def transform(self, data):
    text_vects = []
    text_list = data

    for text in text_list:
      text_vects.append(self.transformtext(data, text))

    return np.matrix(text_vects)

  def transformtext(self, data, text):
    text_list = data
    text_tokenized = word_tokenize(text)
    
    word_vector = np.zeros(len(self.word_index))
    text_tfidf = self.text_tfidf(text)

    for word in text_tokenized:
      if word in self.word_index:
        word_index = self.word_index[word]
        word_vector[word_index] = text_tfidf[word]

    return word_vector

  def text_tfidf(self, text):
    text_tfidf = {}
    text_tf = txf(text)
    num_words = sum(text_tf.values())
    
    average_freq = {k:(float(v)/num_words) for k, v in text_tf.items()}

    for term, tf in average_freq.items():
      text_tfidf[term] = tf*self.idf_dict.get(term, 0)

    return text_tfidf

{'When': 1, 'this': 2, 'first': 1, 'came': 1, 'out': 1, ',': 5, 'my': 1, 'dad': 1, 'brought': 1, 'it': 5, 'home-': 1, 'we': 2, 'were': 1, 'amazed': 1, 'by': 1, 'it-': 1, 'It': 1, 'was': 2, 'so': 1, 'different': 1, 'from': 1, 'anything': 1, 'had': 1, 'seen': 1, 'before': 1, '.': 8, 'I': 3, 'looking': 1, 'for': 2, 'a': 4, 'specific': 1, 'movie': 1, 'last': 1, 'night': 1, 'and': 5, 'found': 1, '``': 2, 'The': 5, 'Mind': 4, "'s": 5, 'Eye': 4, "''": 6, 'again': 1, 'box': 1, 'is': 7, 'falling': 1, 'apart': 1, 'am': 1, 'surprised': 1, 'that': 1, 'the': 7, 'tape': 1, 'still': 2, 'works': 1, '!': 2, 'Although': 1, 'not': 1, "'Finding": 1, 'Nemo': 1, "'": 1, 'quality': 1, 'graphics': 1, 'very': 1, 'good': 1, 'They': 1, 'should': 1, 'sell': 1, 'again-': 1, 'landmark': 1, 'computer': 3, 'animation': 3, 'imagery': 1, 'Highly': 1, 'recommended': 1, '<': 6, 'br': 6, '/': 6, '>': 6, 'This': 2, 'what': 1, ':': 1, 'spectacular': 1, 'odyssey': 1, 'through': 2, 'time': 1, 'Your': 1, 'journey': 1, 'begins'

In [13]:
#test = TFIDF()
#test.fit(train_data)
#test.transform(train_data)
#print(test.index_word)

# Machine Learning Algorithms


## Multinomial Naive Bayes

## SVM

In [28]:
def accuracy(data, predictions):
  data = data.to_numpy()
  correct = 0

  for i in range(len(predictions)):
    if (float(data[i]) == 1.0 and predictions[i][0] == 1.0) or (float(data[i]) == 0.0 and predictions[i][0] == 0.0):
      correct += 1

  return float(correct/len(predictions))

In [26]:
#Implementation of SVM class inspired by 

from cvxopt import matrix, solvers

class SVM():
  def __init__(self, C):
    #C, the stretch parameter
    self.C = C
    self.kernel = self.kernel_lin

  def fit(self, X, y):
    m, n = Xtrain.shape
    y = y.reshape(-1, 1)
    y = y.astype('float')

    K = self.kernel(X, X)

    self.labels = np.unique(y)
    print(self.labels)
    
    recode = y == self.labels[0]
    y[recode] = 1       
    y[~recode] = -1
    y = y * 1.

    assert(X.shape[0] == y.shape[0])

    P = matrix(np.matmul(y,y.T)*K)
    q = matrix(np.ones((m, 1))*-1)
    A = matrix(y.reshape(1, -1))
    b = matrix(np.zeros(1))
    G = matrix(np.vstack((np.eye(m) * -1, np.eye(m))))        
    h = matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
    print(A.size)

    sol = solvers.qp(P,q,G,h,A,b)

    alpha = np.array(sol['x'])
    ind = (alpha > 1e-4).flatten()

    self.sv = X[ind]
    self.sv_y = y[ind]
    self.alpha = alpha[ind]

    b = self.sv_y - np.sum(self.kernel(self.sv, self.sv) * self.alpha * self.sv_y, axis = 0)
    self.b = np.sum(b)/b.size

    self.w = np.sum(self.alpha * self.sv_y * self.sv, axis=0)

  def kernel_lin(self, u, v):
    return np.dot(u, v.T)

  def predict(self, Xtest):
        if self.w is None:
            print("No model constructed.")
            return
        else:
            pred = np.dot(self.w, Xtest.T) + self.b
        ypred = np.sign(pred)
        ypred = ypred.reshape(-1, 1)
        recode = ypred == 1.
        ypred[recode] = self.labels[0]
        ypred[~recode] = self.labels[1]

        return ypred
  

In [27]:
test = SVM(0.5)
print(Xtrain.shape, ytrain.shape)
print(type(Xtrain), type(ytrain))
test.fit(Xtrain, ytrain)

(16000, 100) (16000,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[0. 1.]
(1, 16000)
     pcost       dcost       gap    pres   dres
 0: -8.2724e+03 -2.7384e+04  3e+05  8e+00  4e-14
 1: -4.0221e+03 -2.3545e+04  5e+04  9e-01  4e-14
 2: -3.4825e+03 -1.1878e+04  1e+04  9e-02  2e-14
 3: -3.9835e+03 -5.4920e+03  2e+03  1e-02  2e-14
 4: -4.4290e+03 -4.9621e+03  6e+02  4e-03  3e-14
 5: -4.5536e+03 -4.8130e+03  3e+02  2e-03  3e-14
 6: -4.6062e+03 -4.7497e+03  1e+02  8e-04  3e-14
 7: -4.6358e+03 -4.7137e+03  8e+01  4e-04  3e-14
 8: -4.6558e+03 -4.6896e+03  3e+01  1e-04  3e-14
 9: -4.6650e+03 -4.6788e+03  1e+01  4e-05  3e-14
10: -4.6693e+03 -4.6737e+03  4e+00  8e-06  3e-14
11: -4.6706e+03 -4.6724e+03  2e+00  3e-06  3e-14
12: -4.6712e+03 -4.6717e+03  5e-01  4e-07  3e-14
13: -4.6714e+03 -4.6715e+03  2e-01  1e-07  3e-14
14: -4.6714e+03 -4.6715e+03  3e-02  9e-09  3e-14
15: -4.6715e+03 -4.6715e+03  2e-03  5e-10  3e-14
Optimal solution found.


In [29]:
print(test.w)
print(len(test.w))
print(type(test.w))

model = test.w
pred = test.predict(Xtest)

print(accuracy(test_labs, pred))

[ 1.09860299  1.06714971  0.71838416 -1.7893525   0.48496924  0.55121612
 -0.38773026  6.62810632 -2.64352775 -0.13356428  0.13060877 -0.20675531
  0.16910031 -0.51963857 -0.5359877   1.56531478 -0.7114173   1.3379691
 -0.05398679  2.3681271   0.31052066 -0.63670005  0.29249828 -0.59976748
 -0.61677613 -0.69598712 -0.54869709 -0.08198894  0.43702895  0.48519839
  0.32175558  1.03828884 -0.73032076 -3.7374284   0.58963835  0.44194041
 -0.58440978  0.34077279 -1.5276078   0.03648101 -0.35642904 -0.01243995
  1.3427708  -0.67418524 -2.52516561  0.67117835 -0.2757355  -0.98767435
  2.53281026  0.21987184  0.50628894  0.40405681 -0.83125111  3.70903763
  0.31693085 -0.18680347 -0.24899466  0.13630334 -1.58680841 -0.72839729
  1.60292994  1.31070043  1.17429963 -0.9788435   0.07117167  0.27084579
 -0.88165817  0.96524836  0.44026199 -1.25726275  1.42904085 -0.14707607
 -0.61303662  0.51578482  0.02676161  0.68410148 -1.69459633 -0.61050815
 -0.51148297 -0.77478606  0.81417122 -0.39552029 -0.

AttributeError: ignored

## Deep Neural Network (DNN)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import ModelCheckpoint
from keras.utils.np_utils import to_categorical

n = 80000;
num_classes=2
def dnn(hidden_layers=0):
  model = Sequential()
  model.add(Dense(n, input_shape=train_data_tfidf[1:]))
  model.add(Activation('relu'))

  for i in range(hidden_layers):
    model.add(Dropout(0.5))
    model.add(Dense(units=n))
    model.add(Activation('relu'))

  model.add(Dropout(0.5))
  model.add(Flatten())

  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

  model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
  print(model.summary())

  return model



In [ ]:
history = dict()

for i in range(2):
  print("Training DNN:")
  model = dnn(i)
  history[i] = model.fit(train_data_tfidf, train_labs, batch_size=5, epochs=3, validation_data=(test_data_tfidf, test_labs),shuffle=True)

In [ ]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

SVM = svm.SVC(C=1.0, kernel="linear", degree=3, gamma='auto')
SVM.fit(train_data_tfidf, train_labs)

SVM_pred = SVM.predict(test_data_tfidf)

print("SVM accuracy:", accuracy_score(SVM_pred))